# Symbolic Polar Transformation using Sagemath

In [1]:
from sage import *
import numpy as np
from wireless import sagepolar

In [40]:
def npolar_transform(u,N_P=4/1):
    u = list(u)
    print(len(u), u)
    if len(u) == 1:
        x = u
    elif len(u) > N_P:
        # 처음이 1이고 갈수록 2, 4, 8이 된다.
        # N_P = N/P 값에 따라 AE 담당할 앞부분은 polar 변환하지 않고 뒤집기만 한다.
        u1 = u[0::2]
        u2 = u[1::2]
        x = npolar_transform(u1,N_P) + npolar_transform(u2,N_P)                
    else:
        u1 = u[0::2]
        u2 = u[1::2]
        u1u2 = []
        for u1_i, u2_i in zip(u1, u2):
            u1u2.append(u1_i + u2_i)
        x = npolar_transform(u1u2,N_P) + npolar_transform(u2,N_P)
    return x

def npolar_coding(N=8, P=1):
    """
    Input:
    P=1: AE 입력의 크기임. NPolar의 경우, P=N을 제외하고는 AE를 one-hot vector로 처리하지 않음.
    """    
    N_P = N / P
    u = var('U',n=N)
    y_polar = npolar_transform(u, N_P=N_P)
    #print('y_polar:', y_polar)
    x_polar = []    
    x_polar_idx = []
    ae_polar = y_polar.copy()
    idx = list(range(N))
    for i in range(N_P):
        y_polar_l = y_polar[i::N_P]
        idx_l = idx[i::N_P]
        x_polar.append(y_polar_l)
        x_polar_idx.append(idx_l)
        ae_polar_l = ae_coding(x_polar[-1])
        for i, j in enumerate(x_polar_idx[-1]):
            ae_polar[j] = ae_polar_l[i]
        #print('x_polar:', x_polar)
    return x_polar, x_polar_idx, ae_polar

def ae_coding(x):
    print('AE:', x)
    return x

def bit_forward(x):
    """
    polar encoding에서 사용한 방법을 역으로 수행함
    Polar encoding: x = encoding(x[0::2]) + encoding(x[1::2]) if len(x) > 1
    """
    LN = len(x)
    if LN == 1:
        return x
    else:
        y = x.copy()
        #print(x, y)
        y[0::2] = bit_forward(x[:LN/2])
        y[1::2] = bit_forward(x[LN/2:])
    return y

## n = 4

In [41]:
npolar_coding(N=4, P=1)

4 [U0, U1, U2, U3]
2 [U0 + U1, U2 + U3]
1 [U0 + U1 + U2 + U3]
1 [U2 + U3]
2 [U1, U3]
1 [U1 + U3]
1 [U3]
AE: [U0 + U1 + U2 + U3]
AE: [U2 + U3]
AE: [U1 + U3]
AE: [U3]


([[U0 + U1 + U2 + U3], [U2 + U3], [U1 + U3], [U3]],
 [[0], [1], [2], [3]],
 [U0 + U1 + U2 + U3, U2 + U3, U1 + U3, U3])

In [42]:
npolar_coding(N=4, P=2)

4 [U0, U1, U2, U3]
2 [U0, U2]
1 [U0 + U2]
1 [U2]
2 [U1, U3]
1 [U1 + U3]
1 [U3]
AE: [U0 + U2, U1 + U3]
AE: [U2, U3]


([[U0 + U2, U1 + U3], [U2, U3]], [[0, 2], [1, 3]], [U0 + U2, U2, U1 + U3, U3])

In [43]:
npolar_coding(N=4, P=4)

4 [U0, U1, U2, U3]
2 [U0, U2]
1 [U0]
1 [U2]
2 [U1, U3]
1 [U1]
1 [U3]
AE: [U0, U2, U1, U3]


([[U0, U2, U1, U3]], [[0, 1, 2, 3]], [U0, U2, U1, U3])

In [45]:
bit_forward([U0, U2, U1, U3])

[U0, U1, U2, U3]

## n = 8

In [35]:
npolar_coding(N=8, P=1)

8 [U0, U1, U2, U3, U4, U5, U6, U7]
4 [U0 + U1, U2 + U3, U4 + U5, U6 + U7]
2 [U0 + U1 + U2 + U3, U4 + U5 + U6 + U7]
1 [U0 + U1 + U2 + U3 + U4 + U5 + U6 + U7]
1 [U4 + U5 + U6 + U7]
2 [U2 + U3, U6 + U7]
1 [U2 + U3 + U6 + U7]
1 [U6 + U7]
4 [U1, U3, U5, U7]
2 [U1 + U3, U5 + U7]
1 [U1 + U3 + U5 + U7]
1 [U5 + U7]
2 [U3, U7]
1 [U3 + U7]
1 [U7]
AE: [U0 + U1 + U2 + U3 + U4 + U5 + U6 + U7]
AE: [U4 + U5 + U6 + U7]
AE: [U2 + U3 + U6 + U7]
AE: [U6 + U7]
AE: [U1 + U3 + U5 + U7]
AE: [U5 + U7]
AE: [U3 + U7]
AE: [U7]


([[U0 + U1 + U2 + U3 + U4 + U5 + U6 + U7],
  [U4 + U5 + U6 + U7],
  [U2 + U3 + U6 + U7],
  [U6 + U7],
  [U1 + U3 + U5 + U7],
  [U5 + U7],
  [U3 + U7],
  [U7]],
 [[0], [1], [2], [3], [4], [5], [6], [7]],
 [U0 + U1 + U2 + U3 + U4 + U5 + U6 + U7,
  U4 + U5 + U6 + U7,
  U2 + U3 + U6 + U7,
  U6 + U7,
  U1 + U3 + U5 + U7,
  U5 + U7,
  U3 + U7,
  U7])

In [36]:
npolar_coding(N=8, P=2)

8 [U0, U1, U2, U3, U4, U5, U6, U7]
4 [U0, U2, U4, U6]
2 [U0 + U2, U4 + U6]
1 [U0 + U2 + U4 + U6]
1 [U4 + U6]
2 [U2, U6]
1 [U2 + U6]
1 [U6]
4 [U1, U3, U5, U7]
2 [U1 + U3, U5 + U7]
1 [U1 + U3 + U5 + U7]
1 [U5 + U7]
2 [U3, U7]
1 [U3 + U7]
1 [U7]
AE: [U0 + U2 + U4 + U6, U1 + U3 + U5 + U7]
AE: [U4 + U6, U5 + U7]
AE: [U2 + U6, U3 + U7]
AE: [U6, U7]


([[U0 + U2 + U4 + U6, U1 + U3 + U5 + U7],
  [U4 + U6, U5 + U7],
  [U2 + U6, U3 + U7],
  [U6, U7]],
 [[0, 4], [1, 5], [2, 6], [3, 7]],
 [U0 + U2 + U4 + U6,
  U4 + U6,
  U2 + U6,
  U6,
  U1 + U3 + U5 + U7,
  U5 + U7,
  U3 + U7,
  U7])

In [37]:
npolar_coding(N=8, P=4)

8 [U0, U1, U2, U3, U4, U5, U6, U7]
4 [U0, U2, U4, U6]
2 [U0, U4]
1 [U0 + U4]
1 [U4]
2 [U2, U6]
1 [U2 + U6]
1 [U6]
4 [U1, U3, U5, U7]
2 [U1, U5]
1 [U1 + U5]
1 [U5]
2 [U3, U7]
1 [U3 + U7]
1 [U7]
AE: [U0 + U4, U2 + U6, U1 + U5, U3 + U7]
AE: [U4, U6, U5, U7]


([[U0 + U4, U2 + U6, U1 + U5, U3 + U7], [U4, U6, U5, U7]],
 [[0, 2, 4, 6], [1, 3, 5, 7]],
 [U0 + U4, U4, U2 + U6, U6, U1 + U5, U5, U3 + U7, U7])

In [38]:
npolar_coding(N=8, P=8)

8 [U0, U1, U2, U3, U4, U5, U6, U7]
4 [U0, U2, U4, U6]
2 [U0, U4]
1 [U0]
1 [U4]
2 [U2, U6]
1 [U2]
1 [U6]
4 [U1, U3, U5, U7]
2 [U1, U5]
1 [U1]
1 [U5]
2 [U3, U7]
1 [U3]
1 [U7]
AE: [U0, U4, U2, U6, U1, U5, U3, U7]


([[U0, U4, U2, U6, U1, U5, U3, U7]],
 [[0, 1, 2, 3, 4, 5, 6, 7]],
 [U0, U4, U2, U6, U1, U5, U3, U7])

In [46]:
bit_forward([U0, U4, U2, U6, U1, U5, U3, U7])

[U0, U1, U2, U3, U4, U5, U6, U7]